<a href="https://colab.research.google.com/github/erika024/data-science/blob/main/RAG_GL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#  ADD PATH
# loader = TextLoader('/content/drive/MyDrive/exec ed GL/RAG/AAPL-MDA.txt')
# data = loader.load()

# import whole directory...
loader2 = DirectoryLoader('/content/drive/MyDrive/RAG/MDA Data', glob="*.txt", loader_cls=TextLoader)
data2 = loader2.load()
print(data2[0])


page_content=' Item 7. Management\'s Discussion and Analysis of Financial Condition and Results of Operations See "Management\'s Discussion and Analysis" contained in the Registrant\'s 2002 Annual Report to Shareholders, which information is incorporated herein by reference. Item 7A.' metadata={'source': '/content/drive/MyDrive/RAG/MDA Data/ADP.txt'}


In [16]:
print(data2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)

text_chunks = text_splitter.split_documents(data2)


In [18]:
len(text_chunks)

12597

In [19]:
#get the third chunk
text_chunks[2].page_content

"data-related operations, accounts receivable, accounts payable, fixed assets, payroll, general accounting, financial accounting consolidation, cash management, human resources, tax, legal and real estate. Under these agreements, we reimbursed Eaton for its direct and indirect costs of providing these services until completion of the divestiture, and thereafter, for a limited time, we reimbursed Eaton for its costs plus an additional fee. The transition periods covered by these agreements generally expired on December 29, 2001. The agreements did not necessarily reflect the costs of obtaining these services from unrelated third parties or of providing the applicable services in-house. However, management believed that purchasing these services from Eaton provided an efficient means of obtaining these services during the transition period. Basis of Presentation On June 30, 2000, Eaton substantially completed the transfer of all the assets and related liabilities of its semiconductor equ

In [20]:
len(text_chunks[2].page_content)

1996

In [21]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [23]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers \
                trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.0 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
from transformers import models

In [ ]:
!pip install huggingface_hub -q

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGUF"
model_basename = "llama-2-7b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

In [ ]:
sys_prompt= """
you are a helpful assistant who can go through a document and answer questions
"""

In [ ]:
sys_prompt

In [ ]:
#Import Model
llm = LlamaCpp(
    prompt= sys_prompt,
    streaming = True,
    model_path=model_path,
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

In [ ]:
agent = RetrievalQA.from_chain_type(llm=llm, verbose=True, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}),chain_type_kwargs = {"verbose":True})

In [ ]:
# The year-over-year growth in R&D expense in 2020 was driven primarily by increases in headcount-related expenses.
query ="What drove research and development expense growth in 2020?"

In [ ]:
output = agent.run(query)

In [ ]:
output